In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings

import sklearn.linear_model as sklm
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, log_loss
from sklearn.naive_bayes import GaussianNB

warnings.filterwarnings("ignore")

In [ ]:
# The constant number of possible disease outcomes
# 0: Complete Remission/Response
# 1: Stable Disease
# 2: Progressive Disease
# 3: Partial Remission/Response
# 4: Persistent Disease
OUTCOMES_NUM = 5

# The number of cross-validation train groups
CV_GROUPS = 5

# **Prediction: Vital status**

In [ ]:
def vital_status_train(X, y, alpha, iters, feature_labels, runs = 1, details = True):
    """
    Trains the models to predict the vital status of a patient by the end of cancer treatment.

    :param X: The input matrix (ndarray)
    :param y: The output vector (ndarray)
    :param alpha: The gradient descent coefficient (float)
    :param iters: The number of gradient descent iterations (int)
    :param feature_labels: The list of feature labels (list)
    :param runs: The number of times to train the model (int)
    :param details: True for the full output (bool)
    
    """    
    # Display only the average accuracies
    if not details:
        class_accuracy = train_accuracy = test_accuracy = bayes_accuracy = 0
        
        # Accumulate accuracy statistics for each training batch
        for i in range(runs):              
            # Get training data and train the models
            X_groups, y_groups, X_groups_full, y_groups_full = get_vital_status_data(X, y)
            class_acc = vital_status_manual(X_groups, y_groups, alpha, iters, details)
            tr_acc, ts_acc, bs_acc = vital_status_scikit(X_groups, y_groups, X_groups_full, y_groups_full, \
                                                         feature_labels, details)               
            class_accuracy = class_accuracy + class_acc
            train_accuracy = train_accuracy + tr_acc
            test_accuracy = test_accuracy + ts_acc
            bayes_accuracy = bayes_accuracy + bs_acc
        print("Logistic Regression (manual):")
        print("Average test accuracy: %.2f" % (class_accuracy/runs),"%")
        print("Logistic Regression (scikit):")      
        print("Average train accuracy: %.2f" % (train_accuracy/runs),"%")
        print("Average test accuracy: %.2f" % (test_accuracy/runs),"%")
        print("Bayes (scikit):")
        print("Average test accuracy: %.2f" % (bayes_accuracy/runs),"%") 
    
    # Display the full statistics and graphics
    else:
        for i in range(runs):             
            # Get training data and train the models
            X_groups, y_groups, X_groups_full, y_groups_full = get_vital_status_data(X, y)
            print("Logistic Regression (manual):")
            vital_status_manual(X_groups, y_groups, alpha, iters, details)
            print("Logistic Regression (scikit):") 
            vital_status_scikit(X_groups, y_groups, X_groups_full, y_groups_full, \
                                feature_labels, details)

In [ ]:
def get_vital_status_data(X, y):  
    """
    Returns the cross-validation groups for the vital status training.

    :param X: The input matrix (ndarray)
    :param y: The output vector (ndarray)

    :return X_groups: Balanced cross-validation groups for X (list)
    :return y_groups: Balanced cross-validation groups for y (list)
    :return X_groups_full: Cross-validation groups for X that include all cases (list)
    :return y_groups_full: Cross-validation groups for y that include all cases (list)
        
    """    
    # Separate and shuffle all cases
    dead_full = np.nonzero(y == 0)[0]
    alive_full = np.nonzero(y == 1)[0]
    np.random.shuffle(dead_full)
    np.random.shuffle(alive_full)
    
    # Equate the number of different types for manual model
    num_cases = min(len(dead_full), len(alive_full))
    dead = dead_full[0:num_cases]
    alive = alive_full[0:num_cases]
    
    # Split each type into cross-validation train groups
    dead_full_groups = np.array_split(dead_full, CV_GROUPS)
    alive_full_groups = np.array_split(alive_full, CV_GROUPS)
    dead_groups = np.array_split(dead, CV_GROUPS)
    alive_groups = np.array_split(alive, CV_GROUPS)
    
    # Accumulate the groups in lists
    subsets = []
    subsets_full = []
    for i in range(CV_GROUPS):
        subsets_full.append(np.concatenate((dead_full_groups[i], alive_full_groups[i])))
        np.random.shuffle(subsets_full[i]) 
        subsets.append(np.concatenate((dead_groups[i], alive_groups[i])))      
        np.random.shuffle(subsets[i])

    # Form the cross-validation groups using all possible types
    X_groups = []
    y_groups = []
    X_groups_full = []
    y_groups_full = []
    for i in range(CV_GROUPS):
        X_groups_full.append(X[subsets_full[i],:])
        y_groups_full.append(y[subsets_full[i]])      
        X_groups.append(X[subsets[i],:])
        y_groups.append(y[subsets[i]])
        
    return X_groups, y_groups, X_groups_full, y_groups_full

In [ ]:
def vital_status_manual(X_groups, y_groups, alpha, iters, details = True,  included = [0, 1]):
    """
    Trains the manual model to predict the vital status.

    :param X_groups: The cross-validation groups for X (list)
    :param y_groups: The cross-validation groups for y (list) 
    :param alpha: The gradient descent coefficient (float)
    :param iters: The number of gradient descent iterations (int)
    :param details: True for the full output (bool)
    :param included: The classes to include (list)

    :return: Average logistic regression test accuracy (float)
        
    """
    # Define parameters for the training
    theta = np.ones(X_groups[0].shape[1])
    train_accuracy = test_accuracy = train_f1_score = test_f1_score = 0
    gradient_cost = 0
    test_confusion = np.zeros(shape=(2, 2))

    # Train on the all groups but the test one
    for i in range(CV_GROUPS):
        # Get train/test X and y batches
        X_train, y_train = get_train_groups(X_groups, y_groups, i)
        X_test = X_groups[i]
        y_test = y_groups[i]
        
        # Train the manual model
        theta, J = gradient_descent(X_train, y_train, theta, alpha, iters)
        gradient_cost = gradient_cost + J[-1]
               
        # Get train and test predictions
        sort_predictions = lambda x: 0 if x < 0.5 else 1
        convert_to_log = np.vectorize(sort_predictions)
        train_preds = logistic(np.dot(X_train, theta))
        train_preds = convert_to_log(train_preds)
        test_preds = logistic(np.dot(X_test, theta))
        test_preds = convert_to_log(test_preds)
        
        # Get the statistics
        accuracy, f1 = get_stats(y_train, train_preds)
        train_accuracy = train_accuracy + accuracy
        train_f1_score = train_f1_score + f1
        accuracy, f1 = get_stats(y_test, test_preds)
        test_accuracy = test_accuracy + accuracy
        test_f1_score = test_f1_score + f1
        test_confusion = test_confusion + confusion_matrix(y_test, test_preds)
    
    # Display the full statistics and graphics
    if details:
        print("Average train final cost: %.2f" % (gradient_cost/CV_GROUPS))
        print("Average train accuracy: %.2f" % (train_accuracy/CV_GROUPS),"%")
        print("Average train F1 score: %.2f" % (train_f1_score/CV_GROUPS))
        print("Average test accuracy: %.2f" % (test_accuracy/CV_GROUPS),"%")
        print("Average test F1 score: %.2f" % (test_f1_score/CV_GROUPS))
        create_heatmap((test_confusion/CV_GROUPS).astype('int'), included)
        
    return test_accuracy/CV_GROUPS

In [ ]:
def vital_status_scikit(X_groups, y_groups, X_groups_full, y_groups_full, feature_labels, details = True):
    """
    Trains the scikit model to predict the vital status.

    :param X_groups: The cross-validation groups for reduced X (list)
    :param y_groups: The cross-validation groups for reduced y (list) 
    :param X_groups_full: The cross-validation groups for full X (list)
    :param y_groups_full: The cross-validation groups for full y (list) 
    :param feature_labels: The list of feature labels (list)
    :param details: True for the full output (bool)
    
    :return: Average logistic regression train accuracy (float)
    :return: Average logistic regression test accuracy (float)
    :return: Average Bayes test accuracy (float)
        
    """    
    # Define parameters for the training
    train_accuracy = test_accuracy = bayes_accuracy = 0
    train_cost = test_cost = 0
    train_f1_score = test_f1_score = bayes_f1_score = 0
    test_weights = np.zeros(X_groups[0].shape[1] - 1)
    test_confusion = np.zeros(shape=(2, 2)) 
    
    for i in range(CV_GROUPS):
        # Get train/test X and y batches
        X_train, y_train = get_train_groups(X_groups, y_groups, i)
        X_train_full, y_train_full = get_train_groups(X_groups_full, y_groups_full, i)
        X_test = X_groups[i]
        y_test = y_groups[i] 
        X_test_full = X_groups_full[i]
        y_test_full = y_groups_full[i] 
        
        # Train the Logistic Regression and Gaussian Bayes models
        lr = sklm.LogisticRegression(solver='liblinear', class_weight = 'balanced')
        lr.fit(X_train_full[:,1:], y_train_full)
        gnb = GaussianNB()
        gnb.fit(X_train[:,1:], y_train)
        
        # Get train and test predictions
        train_preds = lr.predict(X_train_full[:,1:])
        test_preds = lr.predict(X_test_full[:,1:])
        gnb_preds = gnb.predict(X_test[:,1:])
        
        # Get the statistics
        bayes_accuracy = bayes_accuracy + accuracy_score(y_test, gnb_preds)*100
        bayes_f1_score = bayes_f1_score + f1_score(y_test, gnb_preds)       
        train_accuracy = train_accuracy + accuracy_score(y_train_full, train_preds)*100
        train_f1_score = train_f1_score + f1_score(y_train_full, train_preds)
        train_cost = train_cost + log_loss(y_train_full, train_preds)        
        test_accuracy = test_accuracy + accuracy_score(y_test_full, test_preds)*100
        test_f1_score = test_f1_score + f1_score(y_test_full, test_preds)
        test_cost = test_cost + log_loss(y_test_full, test_preds)        
        test_weights = np.add(test_weights, lr.coef_[0])
        test_confusion = test_confusion + confusion_matrix(y_test_full, test_preds)

    # Get the labels with the highest average absolute weights
    actual_labels, actual_weights = get_actual_weights(feature_labels[1:], list(test_weights))    
    test_weights_abs = list(np.abs(actual_weights))
    test_weights_sorted = test_weights_abs.copy()
    test_weights_sorted.sort(reverse = True)
    
    # Display the full statistics and graphics
    if details:
        print("Average train final cost: %.2f" % (train_cost/CV_GROUPS))
        print("Average train accuracy: %.2f" % (train_accuracy/CV_GROUPS),"%")
        print("Average train F1 score: %.2f" % (train_f1_score/CV_GROUPS))
        print("Average test final cost: %.2f" % (test_cost/CV_GROUPS))
        print("Average test accuracy: %.2f" % (test_accuracy/CV_GROUPS),"%")
        print("Average test F1 score: %.2f" % (test_f1_score/CV_GROUPS))
        print("Average Gaussian Naive Bayes accuracy: %.2f" % (bayes_accuracy/CV_GROUPS),"%")
        print("Average Gaussian Naive Bayes F1 score: %.2f" % (bayes_f1_score/CV_GROUPS))
        create_heatmap((test_confusion/CV_GROUPS).astype('int'), [0, 1])
        print()
        print("Highest average weights (absolute values):")
        for i in range(len(test_weights_sorted[:3])):
            index = test_weights_abs.index(test_weights_sorted[i])
            print(actual_labels[index], ': %.2f' % test_weights_sorted[i])
            
    return train_accuracy/CV_GROUPS, test_accuracy/CV_GROUPS, bayes_accuracy/CV_GROUPS

# **Prediction: Life expectancy (in days)**

In [ ]:
def life_expectancy_train(X, y, alpha, iters):
    """
    Trains the models to predict the life expectancy of a patient in days.

    :param X: The input matrix (ndarray)
    :param y: The output vector (ndarray)
    :param alpha: The gradient descent coefficient (float)
    :param iters: The number of gradient descent iterations (int)
    
    """  
    train_accuracy = test_accuracy = final_cost = 0
    
    # Get cross-validation grouped X and y data
    X_groups, y_groups = get_life_expectancy_data(X, y)

    # Perform k-fold cross-validation and record the results 
    for i in range(CV_GROUPS):
        X_test = X_groups[i]
        y_test = y_groups[i]
        X_train, y_train = get_train_groups(X_groups, y_groups, i)

        # Scikit model training
        lr = sklm.LinearRegression()
        lr.fit(X_train, y_train)
        train_preds = lr.predict(X_train)
        test_preds = lr.predict(X_test)

        # Manual training, only cost
        theta_vec_equations, cost = gradient_descent_v2(normalize_v2(X_train), y_train, alpha, iters)   
        final_cost = final_cost + cost[-1]

        # Calculate the train accuracy within the 180 days margin
        match = 0
        for i in range(len(y_train)):
            if  train_preds[i] <= y_train[i] + 180 and train_preds[i] >= y_train[i] - 180:
                match += 1
        train_accuracy = train_accuracy + match/len(y_train)*100 

        # Calculate the test accuracy within the 180 days margin
        match = 0
        for i in range(len(y_test)):
            if  test_preds[i] <= y_test[i] + 180 and test_preds[i] >= y_test[i] - 180:
                match += 1
        test_accuracy = test_accuracy + match/len(y_test)*100 

    print("Manual model:")
    print("Average final cost: %.2f" % (final_cost/CV_GROUPS))
    print("Mean in y: %.2f" % np.mean(y))
    print("Variance in y: %.2f" % np.var(y))
    print()
    print("Scikit model:")
    print("Average train accuracy: %.2f" % (train_accuracy/CV_GROUPS),"%")
    print("Average test accuracy: %.2f" % (test_accuracy/CV_GROUPS),"%")

In [ ]:
def get_life_expectancy_data(X, y):    
    """
    Returns the cross-validation groups for the life expectancy training.

    :param X: The input matrix (ndarray)
    :param y: The output vector (ndarray)

    :return X_groups: Cross-validation groups for X (list)
    :return y_groups: Cross-validation groups for y (list)
        
    """  
    # Get all deceased cases
    death_indices = np.arange(len(y))
    np.random.shuffle(death_indices)
    
    dead_groups = np.array_split(death_indices, CV_GROUPS)

    X_groups = []
    y_groups = []
    for i in range(CV_GROUPS):
        X_groups.append(X[dead_groups[i],:])
        y_groups.append(y[dead_groups[i]])
    return X_groups, y_groups

# **Prediction: Cancer outcome**

In [ ]:
def outcome_train(X, y, included, lambda_val, feature_labels, \
                  alpha = 0.00000001, iters = 2000, runs = 1, details = True):
    """
    Trains the models to predict the outcome of the disease.

    :param X: The input matrix (ndarray)
    :param y: The output vector (ndarray)
    :param lambda_val: Regularization parameter (float)
    :param feature_labels: The feature labels (list)
    :param alpha: The gradient descent coefficient (float)
    :param iters: The number of gradient descent iterations (int)
    :param runs: The number of times to train the model (int)
    :param details: True for the full output (bool)
        
    """  
    num_classes = len(included)
    
    if details == True:
        for i in range(runs):    
            X_groups, y_groups = get_outcome_data(X, y, included)

            print("Manual model:")
            if num_classes == 2:
                for i in range(len(y_groups)):
                    y_groups[i] = np.where(y_groups[i] == included[0], 0, 1)                    
                print("Logistic regression:")
                # Reuse the vital status training implementation for binary classification
                vital_status_manual(X_groups, y_groups, alpha, iters, included = included)
                print()

            print("One vs all:")  
            outcome_manual(X_groups, y_groups, included, lambda_val)
            print()
            
            print("Scikit model:")
            outcome_scikit(X_groups, y_groups, included, feature_labels)
            plot_lambdas(X_groups, y_groups, included)
    else:
        acc_regression = acc_classifier = acc_scikit = acc_bayes = 0
        
        for i in range(runs):  
            X_groups, y_groups = get_outcome_data(X, y, included)

            if num_classes == 2:
                for i in range(len(y_groups)):
                    y_groups[i] = np.where(y_groups[i] == included[0], 0, 1)                    
                acc_regression = acc_regression + vital_status_manual(X_groups, y_groups, alpha, iters, details) 
            acc_classifier = acc_classifier + outcome_manual(X_groups, y_groups, included, lambda_val, details)
            test_scikit, test_bayes = outcome_scikit(X_groups, y_groups, included, feature_labels, details)
            acc_scikit = acc_scikit + test_scikit       
            acc_bayes = acc_bayes + test_bayes    
                
        if num_classes == 2:
            print("Logistic regression:")
            print("Average test accuracy: %.2f" % (acc_regression/runs),"%")
        print("One vs all model:")
        print("Average test accuracy: %.2f" % (acc_classifier/runs),"%")
        print("Scikit model:")
        print("Average test accuracy: %.2f" % (acc_scikit/runs),"%")
        print("Bayes model:")
        print("Average test accuracy: %.2f" % (acc_bayes/runs),"%")

In [ ]:
def get_outcome_data(X, y, included):       
    """
    Returns the cross-validation groups for the outcome training.

    :param X: The input matrix (ndarray)
    :param y: The output vector (ndarray)
    :param included: The classes to include (list)
    
    :return X_groups: The cross-validation groups for X (list)
    :return y_groups: The cross-validation groups for y (list) 
        
    """  
    # Get the number of samples to take from each class
    data = []
    num_cases = 0
    num_classes = len(included)
    for i in range(OUTCOMES_NUM): 
        if i in included:
            cases = np.nonzero(y == i)[0]
            if (num_cases):
                num_cases = min(num_cases, len(cases))
            else:
                num_cases = len(cases)
            data.append(cases) 
    
    # Truncate to balance
    for i in range(num_classes):
        np.random.shuffle(data[i])
        data[i] = data[i][0:num_cases]
    
    # Break into balanced groups
    batch_groups = []
    for i in range(num_classes):
        batch_groups.append(np.array_split(data[i], CV_GROUPS))

    # Put batches together 
    outcome_groups = []
    for i in range(CV_GROUPS):
        batch = []
        for j in range(num_classes):
            batch = np.concatenate([batch, batch_groups[j][i]])
        np.random.shuffle(batch)
        outcome_groups.append(batch.astype('int'))
    
    X_groups = []
    y_groups = []
    for i in range(CV_GROUPS):
        X_groups.append(X[outcome_groups[i],:])
        y_groups.append(y[outcome_groups[i]])
        
    return X_groups, y_groups

In [ ]:
def outcome_manual(X_groups, y_groups, included, lambda_val, details = True):
    """
    Trains the manual model to predict the outcome of the disease.
    
    :param X_groups: The cross-validation groups for X (list)
    :param y_groups: The cross-validation groups for y (list) 
    :param included: The classes to include (list)
    :param lambda_val: Regularization parameter (float)
    :param details: True for the full output (bool)
    
    :return: average test accuracy for one vs all model
        
    """ 
    train_accuracy = test_accuracy = train_f1_score = test_f1_score = 0 
    num_classes = len(included)
    test_confusion = np.zeros(shape=(len(included), len(included)))

    target_names = []
    for i in included:
        target_names.append('class ' + str(i))

    for i in range(CV_GROUPS):
        X_test = X_groups[i]
        y_test = y_groups[i]
        X_train, y_train = get_train_groups(X_groups, y_groups, i)
        
        weight_vectors, intercepts = train_one_vs_all(X_train[:,1:], y_train, included, lambda_val)
        train_preds = predict_one_vs_all(X_train[:,1:], weight_vectors, intercepts)
        test_preds  = predict_one_vs_all(X_test[:,1:],  weight_vectors, intercepts)

        accuracy, f1 = get_stats(y_train, train_preds)
        train_accuracy = train_accuracy + accuracy
        train_f1_score = train_f1_score + f1
        
        accuracy, f1 = get_stats(y_test, test_preds)
        test_accuracy = test_accuracy + accuracy
        test_f1_score = test_f1_score + f1
        test_confusion = test_confusion + confusion_matrix(list(y_test), list(test_preds))
        
    if details:
        print("Average train accuracy: %.2f" % (train_accuracy/CV_GROUPS),"%")
        print("Average train F1 score: %.2f" % (train_f1_score/CV_GROUPS))

        print("Average test accuracy: %.2f" % (test_accuracy/CV_GROUPS),"%")
        print("Average test F1 score: %.2f" % (test_f1_score/CV_GROUPS))   
        create_heatmap((test_confusion/CV_GROUPS).astype('int'), included)
    return test_accuracy/CV_GROUPS

In [ ]:
def outcome_scikit(X_groups, y_groups, included, feature_labels, details = True):
    """
    Trains the scikit model to predict the outcome of the disease.
    
    :param X_groups: The cross-validation groups for X (list)
    :param y_groups: The cross-validation groups for y (list) 
    :param included: The classes to include (list)
    :param feature_labels: The feature labels (list)
    :param details: True for the full output (bool)
    
    :return: Average logistic regression accuracy
    :return: Average Bayes accuracy
        
    """ 
    train_accuracy = test_accuracy = train_f1_score = test_f1_score = bayes_f1_score = test_bayes = 0
    test_weights = np.zeros(X_groups[0].shape[1] - 1)
    test_confusion = np.zeros(shape=(len(included), len(included)))
    
    target_names = []
    for i in included:
        target_names.append('class ' + str(i))
    
    for i in range(CV_GROUPS):
        X_test = X_groups[i]
        y_test = y_groups[i]
        X_train, y_train = get_train_groups(X_groups, y_groups, i)
        
        lr = sklm.LogisticRegression()
        lr.fit(X_train[:,1:], y_train.astype('int'))
        train_preds = lr.predict(X_train[:,1:])
        test_preds = lr.predict(X_test[:,1:])    
        
        # Naive Bayes model
        gnb = GaussianNB()
        gnb.fit(X_train[:,1:], y_train.astype('int'))       
        gnb_preds = gnb.predict(X_test[:,1:])
        
        train_accuracy = train_accuracy + np.mean(train_preds == y_train)*100
        test_accuracy = test_accuracy + np.mean(test_preds == y_test)*100       
        test_bayes = test_bayes + accuracy_score(list(y_test), list(gnb_preds))*100
        train_class_report = classification_report(list(y_train), list(train_preds), \
                                              target_names = target_names, output_dict=True)
        test_class_report = classification_report(list(y_test), list(test_preds), \
                                              target_names = target_names, output_dict=True)     
        bayes_class_report = classification_report(list(y_test), list(gnb_preds), \
                                                   target_names = target_names, output_dict=True)

        # Calculate average test f1-score
        train_f1_score = train_f1_score + train_class_report['weighted avg']['f1-score']
        test_f1_score = test_f1_score + test_class_report['weighted avg']['f1-score']
        bayes_f1_score = bayes_f1_score + bayes_class_report['weighted avg']['f1-score']
        test_weights = np.add(test_weights, lr.coef_[0])
        test_confusion = test_confusion + confusion_matrix(list(y_test), list(test_preds))
    
    actual_labels, actual_weights = get_actual_weights(feature_labels[1:], list(test_weights))   
    test_weights_abs = list(np.abs(actual_weights))
    test_weights_sorted = test_weights_abs.copy()
    test_weights_sorted.sort(reverse = True)
    
    if details:
        print("Average train accuracy: %.2f" % (train_accuracy/CV_GROUPS),"%")
        print("Average train F1 score: %.2f" % (train_f1_score/CV_GROUPS))
        print("Average test accuracy: %.2f" % (test_accuracy/CV_GROUPS),"%")
        print("Average test F1 score: %.2f" % (test_f1_score/CV_GROUPS))
        print("Average Gaussian Naive Bayes accuracy: %.2f" % (test_bayes/CV_GROUPS),"%")
        print("Average Gaussian Naive Bayes F1 score: %.2f" % (bayes_f1_score/CV_GROUPS))
        create_heatmap((test_confusion/CV_GROUPS).astype('int'), included)
        print()
        print("Highest average weights (absolute values!):")
        for i in range(len(test_weights_sorted[:3])):
            index = test_weights_abs.index(test_weights_sorted[i])
            print(actual_labels[index], ': %.2f' % test_weights_sorted[i])
    return test_accuracy/CV_GROUPS, test_bayes/CV_GROUPS

# **Helpers: Training**

In [ ]:
def train_one_vs_all(X, y, included, lambda_val):
    """
    Trains a one vs. all logistic regression
    
    :param X: The input matrix (ndarray)
    :param y: Label vector (ndarray)
    :param included: Indices of the outcomes included in training (ndarray)
    :param lambda_val: Regularization parameter (float)

    :return weight_vectors: Weight vector matrix (ndarray)
    :return intercepts: Intercept vector matrix (ndarray)                     
            
    """
    num_classes = len(included)
    weight_vectors = np.zeros((X.shape[1], num_classes))
    intercepts = np.zeros(num_classes) 

    for i in range(len(included)):
        if included[i] in y:
            y_c = (y == included[i]).astype(int)
            weight_vectors[:, i], intercepts[i] = train_logistic_regression(X, y_c, lambda_val)
    return weight_vectors, intercepts

In [ ]:
def predict_one_vs_all(X, weight_vectors, intercepts):
    """
    Predicts one vs. all logistic regression
    
    :param X: The input matrix (ndarray)
    :param weight_vectors: Weight vector matrix (ndarray)
    :param intercepts: Intercept vector matrix (ndarray)     
                       
    :return predictions: Prediction vector (ndarray) 
    
    """    
    predictions = np.argmax(np.add(np.dot(X,weight_vectors),intercepts), 1)
    return predictions

In [ ]:
def train_logistic_regression(X, y, lambda_val):
    """
    Trains a regularized logistic regression model
    
    :param X: The input matrix (ndarray)
    :param y: Label vector (ndarray)
    :param lambda_val: Regularization parameter (float)

    :return weights: Weight vector (ndarray)
    :return intercept: Intercept parameter (float)   
    
    """
    model = sklm.LogisticRegression(C=2./lambda_val, solver='lbfgs')

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        model.fit(X, y)

    weight_vector = model.coef_.ravel()
    intercept = model.intercept_
    return weight_vector, intercept

In [ ]:
def logistic(z):
    """
    The logistic function
    
    :param z: Array to be converted (ndarray)
    
    :return p: logistic(z) entrywise of the same shape (ndarray)
    
    """
    ones_z = np.ones(z.shape)
    exp = np.full(z.shape, np.e)
    denum = ones_z.copy() + np.power(exp, -z)
    p = np.divide(ones_z, denum)
    return p

In [ ]:
def nll_cost_function(X, y, theta):
    """
    Computes the negative log liklihood (nll) cost function for a particular data 
    set and hypothesis (weight vector)
    
    :param X: The input matrix (ndarray)
    :param y: Label vector (ndarray)
    :param theta: Initial parameter vector (ndarray)
    
    :return cost: The value of the cost function (float)
    
    """
    cost = 0
    h = logistic(np.dot(X,theta)).astype('int')
    cost = -np.dot(y, np.log(h)) - np.dot((1-y), np.log(1 - h))
    return cost

In [ ]:
def gradient_descent(X, y, theta, alpha, iters):
    """
    Fits a logistic regression model by gradient descent.
    
    :param X: The input matrix (ndarray)
    :param y: Label vector (ndarray)
    :param theta: Initial parameter vector (ndarray)
    :param alpha: Step size (float)
    :param iters: Number of iterations (int)
    
    :return theta: Learned parameter vector (ndarray)
    :return J_history: Cost function in iteration (ndarray)
        
    """
    J_history = np.zeros(iters)
    
    for i in range(iters):
        d_J = 2*(np.dot(X.T, np.subtract(logistic(np.dot(X,theta)),y)))
        theta = theta - (alpha*d_J)
        J_history[i] = nll_cost_function(X, y, theta)
    return theta, J_history

In [ ]:
def gradient_descent_v2(X, y, alpha, iters, theta=None):
    """
    Gradient descent function for the life expectancy training.
    
    :param X: The input matrix (ndarray)
    :param y: Label vector (ndarray)
    :param alpha: Step size (float)
    :param iters: The number of gradient descent iterations (int)
    :param theta: Initial parameter vector (ndarray)

    :return theta: Learned parameter vector (ndarray)
    :return J_history: Cost function in iteration (ndarray)                    
            
    """
    m,n = X.shape
    if theta is None:
        theta = np.ones(n)  
    J_history = np.zeros(iters)

    for i in range(0, iters):       
        theta = theta - np.dot(alpha*X.T, np.dot(X, theta) - y)
        J_history[i] = cost_function_v2(X, y, theta)    
    return theta, J_history

In [ ]:
def cost_function_v2(X, y, theta):
    """
    Calculates the cost for the life expectancy training.
    
    :param X: The input matrix (ndarray)
    :param y: Label vector (ndarray)
    :param theta: Initial parameter vector (ndarray)

    :return cost: The training cost (float)                 
            
    """
    cost = 0
    diff = (np.dot(X,theta)-y).T
    diff = np.where(abs(diff) < 30, 0, diff)
    cost = 0.5*np.dot(diff, diff)
    return cost

In [ ]:
def normalize_v2(M):
    """
    Normalizes the data of the input matrix.
    
    :param M: The input matrix (ndarray)

    :return norm_M: The normalized matrix (ndarray)                 
            
    """
    norm_M = M.copy()
    mean = np.mean(M[:, 1:].copy(), axis = 0)
    std = np.array(np.std(M[:, 1:].copy(), axis = 0))
    norm_M[:,1:] = np.divide(np.subtract(norm_M[:,1:], mean), std)
    return norm_M

# **Helpers: Statistics**

In [ ]:
def get_train_groups(X_groups, y_groups, test_group):
    """
    Returns the assembled train set without the test group

    :param X_groups: The cross-validation groups for reduced X (list)
    :param y_groups: The cross-validation groups for reduced y (list) 
    :param test_group: The test group index (int)
    
    :return X_train: The assembled train batch for X
    :return y_train: The assembled train batch for y
        
    """  
    X_train = []
    y_train = []
    for i in range(CV_GROUPS):
        if (i != test_group):
            if len(X_train) == 0:
                X_train = np.array(X_groups[i])
                y_train = np.array(y_groups[i])
            else:
                X_train = np.concatenate((X_train, X_groups[i]))
                y_train = np.concatenate((y_train, y_groups[i]))
    return X_train, y_train

In [ ]:
def get_stats(trues, predictions):
    """
    Calculates the accuracy and F1 score, given the expected/predicted values.

    :param trues: The expected values (list)
    :param predictions: The predicted values (list) 
    
    :return accuracy: The prediction accuracy
    :return y_train: The prediction F1 score
        
    """  
    stat_zip = zip(predictions, trues)
    
    match = tp = fp = fn = 0
    for x in stat_zip:
        if x[0] != x[1] and x[0] == 1:
            fp = fp + 1
        elif x[0] != x[1] and x[0] == 0:
            fn = fn + 1
        elif x[0] == x[1]:
            match = match + 1
            if x[0] == 1:
                tp = tp + 1
    
    accuracy = match/len(trues)*100   
    precision = 0 if (tp + fp == 0) else tp/(tp + fp)
    recall = 0 if (tp + fn == 0) else tp/(tp + fn)
    f1 = 0 if (precision + recall == 0) else 2*precision*recall/(precision + recall)
    
    return accuracy, f1

In [ ]:
def print_num_samples(y):
    """
    Prints out the number of samples of each class.

    :param y: The y values (list)

    """  
    for i in range(OUTCOMES_NUM): 
        print("Samples of class", i, ":", len(np.nonzero(y == i)[0]))
    print()

In [ ]:
def get_actual_weights(feature_labels, weights):
    """
    Gets the average (compressed) weights to assess feature importance. 

    :param feature_labels: The feature labels (list)
    :param weights: The feature weights (list)
    
    :return actual_labels: The labels applicable to the set (list)
    :return weights: The compressed feature weights (list)

    """  
    actual_labels = []

    compressed_labels = ['histological_grade', 'clinical_stage', 'tumor_grade', 'histological_type', 'tumor_stage']
    compressed_indices = []
    for i in range(len(compressed_labels)):
        compressed_indices.append([])

    # extract actual labels
    for i in range(len(feature_labels)):  
        success = False
        for j in range(len(compressed_labels)):
            if feature_labels[i].startswith(compressed_labels[j]):
                compressed_indices[j].append(i)
                if compressed_labels[j] not in actual_labels:
                    actual_labels.append(compressed_labels[j])
                success = True
                break
        if not success:
            actual_labels.append(feature_labels[i])
    compressed_indices.sort(reverse = True)

    # compress weights
    for i in range(len(compressed_indices)):
        if compressed_indices[i]:
            num_elem = len(compressed_indices[i]) 
            start = compressed_indices[i][0]
            finish = compressed_indices[i][0] + num_elem
            weights[start : finish] = [np.mean( weights[start : finish])]
    return actual_labels, weights

# **Helpers: Visual**

In [ ]:
def create_heatmap(cnf_matrix, included):
    """
    Creates the heatmap of the confusion matrix.
    
    :param cnf_matrix: The confusion matrix (ndarray)
    :param included: The classes to include (list)
    
    """
    fig, ax = plt.subplots()
    tick_marks = np.arange(len(included))
    plt.xticks(tick_marks)
    plt.yticks(tick_marks)

    # create heatmap
    sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
    ax.xaxis.set_label_position("top")
    
        
    labels_X = [item.get_text() for item in ax.get_xticklabels()]
    labels_Y = [item.get_text() for item in ax.get_yticklabels()]
    for i in range(len(included)):
        labels_X[i] = included[i]
        labels_Y[i] = included[i]
    ax.set_xticklabels(labels_X)
    ax.set_yticklabels(labels_Y)

    
    plt.tight_layout()
    plt.title('Confusion matrix', y=1.1)
    plt.ylabel('True label')
    plt.xlabel('Predicted label') 
    plt.show()

In [ ]:
def plot_lambdas(X_groups, y_groups, included):
    """
    Plots the accuracy of the one-vs-all model for several different lambdas.
    
    :param X_groups: The cross-validation groups for X (list)
    :param y_groups: The cross-validation groups for y (list) 
    :param included: The number of gradient descent iterations (int)
    
    """
    train_accuracy = test_accuracy = 0
    num_classes = len(included)

    lambda_vals = 10.0 ** np.linspace(-5, 5, 11)
    test_acc = np.zeros(lambda_vals.size)
    train_acc = np.zeros(lambda_vals.size)

    for j in range(lambda_vals.size): 
        train_accuracy = test_accuracy = 0

        for i in range(CV_GROUPS):
            X_test = X_groups[i]
            y_test = y_groups[i]
            X_train, y_train = get_train_groups(X_groups, y_groups, i)

            weight_vectors, intercepts = train_one_vs_all(X_train, y_train, included, lambda_vals[j])
            train_preds = predict_one_vs_all(X_train, weight_vectors, intercepts)
            test_preds  = predict_one_vs_all(X_test, weight_vectors, intercepts)

            train_accuracy = train_accuracy + np.mean(train_preds == y_train)*100
            test_accuracy = test_accuracy + np.mean(test_preds == y_test)*100 

        train_acc[j] = train_accuracy/CV_GROUPS
        test_acc[j] = test_accuracy/CV_GROUPS

    plt.xlabel('lambda (log10)')
    plt.ylabel('accuracy')
    plt.xscale('log')
    plt.plot(lambda_vals, train_acc, 'bo', linestyle='dashed')
    plt.plot(lambda_vals, test_acc, 'go', linestyle='dashed')
    plt.legend(('train', 'test')) 
    plt.show()